In [1]:
from app.shared.config import settings


/Users/parsa/Desk/projects/university/slmops-project/application/slmops-application-qa/backend/app/shared/config.py:115: UserWarning: The value of SECRET_KEY is "changethis", for security, please change it, at least for deployments.
  warnings.warn(message, stacklevel=1)
/Users/parsa/Desk/projects/university/slmops-project/application/slmops-application-qa/backend/app/shared/config.py:115: UserWarning: The value of FIRST_SUPERUSER_PASSWORD is "changethis", for security, please change it, at least for deployments.
  warnings.warn(message, stacklevel=1)


In [29]:
from langchain_openai import ChatOpenAI
import nest_asyncio
from langchain.prompts import PromptTemplate,ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda 
nest_asyncio.apply()
import time
def simple_function(input: str):
    time.sleep(1)
    yield "hi"


llm = model = ChatOpenAI(model="gpt-4o-mini")



In [30]:
from langchain_core.output_parsers import JsonOutputParser


async def _extract_country_names_streaming(input_stream):
    """A function that operates on input streams."""
    country_names_so_far = set()

    async for input in input_stream:
        if not isinstance(input, dict):
            continue

        if "countries" not in input:
            continue

        countries = input["countries"]

        if not isinstance(countries, list):
            continue

        for country in countries:
            name = country.get("name")
            if not name:
                continue
            if name not in country_names_so_far:
                yield name
                country_names_so_far.add(name)


chain = model | JsonOutputParser() | _extract_country_names_streaming

async for text in chain.astream(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`",
):
    print(text, end="|", flush=True)

France|Spain|Japan|

In [32]:
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

vectorstore = Chroma.from_texts(
    ["harrison worked at kensho", "harrison likes spicy food"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

chunks = [chunk for chunk in retriever.stream("where did harrison work?")]
chunks

retrieval_chain = (
    {
        "context": retriever.with_config(run_name="Docs"),
        "question": RunnablePassthrough(),
    }
    | prompt
    | model
    | StrOutputParser()
)

for chunk in retrieval_chain.stream(
    "Where did harrison work? " "Write 3 made up sentences about this place."
):
    print(chunk, end="|", flush=True)


|H|arrison| worked| at| Kens|ho|.| Kens|ho| is| known| for| its| innovative| approach| to| technology|,| blending| artificial| intelligence| with| user|-friendly| interfaces|.| The| office| is| filled| with| vibrant| artwork| that| inspires| creativity| among| its| employees|.| Additionally|,| Kens|ho| hosts| weekly| brainstorming| sessions| where| team| members| can| pitch| their| ideas| in| a| collaborative| environment|.||

In [33]:
events = []
async for event in retrieval_chain.astream_events("Where did harrison work? " "Write 3 made up sentences about this place.", version="v2"):
    events.append(event)
    print(event)
    

{'event': 'on_chain_start', 'data': {'input': 'Where did harrison work? Write 3 made up sentences about this place.'}, 'name': 'RunnableSequence', 'tags': [], 'run_id': 'e35da027-0117-4539-bac4-d9d769525d1c', 'metadata': {}, 'parent_ids': []}
{'event': 'on_chain_start', 'data': {}, 'name': 'RunnableParallel<context,question>', 'tags': ['seq:step:1'], 'run_id': '143955e2-77e0-41d2-95f6-501dead1d1e8', 'metadata': {}, 'parent_ids': ['e35da027-0117-4539-bac4-d9d769525d1c']}
{'event': 'on_retriever_start', 'data': {'input': {'query': 'Where did harrison work? Write 3 made up sentences about this place.'}}, 'name': 'Docs', 'tags': ['map:key:context', 'Chroma', 'OpenAIEmbeddings'], 'run_id': '42e252c2-c32b-418e-9d1b-062586e93a8a', 'metadata': {'ls_retriever_name': 'vectorstore', 'ls_vector_store_provider': 'Chroma', 'ls_embedding_provider': 'OpenAIEmbeddings'}, 'parent_ids': ['e35da027-0117-4539-bac4-d9d769525d1c', '143955e2-77e0-41d2-95f6-501dead1d1e8']}
{'event': 'on_chain_start', 'data': {

In [35]:
num_events = 0

async for event in retrieval_chain.astream_events("Where did harrison work? " "Write 3 made up sentences about this place.", version="v2"):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        print(
            f"Chat model chunk: {repr(event['data']['chunk'].content)}",
            flush=True,
        )
    # if kind == "on_parser_stream":
    #     print(f"Parser chunk: {event['data']['chunk']}", flush=True)
    num_events += 1
    if num_events > 60:
        # Truncate the output
        print("...")
        break

Chat model chunk: ''
Chat model chunk: 'H'
Chat model chunk: 'arrison'
Chat model chunk: ' worked'
Chat model chunk: ' at'
Chat model chunk: ' Kens'
Chat model chunk: 'ho'
Chat model chunk: '.'
Chat model chunk: ' Kens'
Chat model chunk: 'ho'
Chat model chunk: ' is'
Chat model chunk: ' known'
Chat model chunk: ' for'
Chat model chunk: ' its'
Chat model chunk: ' innovative'
Chat model chunk: ' approach'
...


In [ ]:

async for event in retrieval_chain.astream_events("Where did harrison work? " "Write 3 made up sentences about this place.", version="v2"):
    if event["event"] == "on_llm_new_token":
    print(event)